In [ ]:
from vision.preprocess import preprocess
from models.model_loader import load_model
from control.robot_moving import RobotMoving
from control.area_finder import WorkingAreaFind
from gripper.gripper_control import setup_gripper, grip, release
from utils.s3_uploader import start_s3_uploader

from jetbot import Robot, Camera, bgr8_to_jpeg
import ipywidgets as widgets
from IPython.display import display
import threading

# ─────────────────────────────────────────────────────
# 1. JetBot 하드웨어 초기화
# ─────────────────────────────────────────────────────
robot   = Robot()
camera  = Camera()
gripper = setup_gripper()
print("✅ 로봇, 카메라, 그리퍼 초기화 완료")

# ─────────────────────────────────────────────────────
# 2. AI 모델 로드
# ─────────────────────────────────────────────────────
model, device, mean, std = load_model("best.pth")
print("✅ 모델 로드 완료")

# ─────────────────────────────────────────────────────
# 3. UI 구성
# ─────────────────────────────────────────────────────
image_widget = widgets.Image(format='jpeg', width=224, height=224)
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = widgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = widgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

speed_gain_slider = widgets.FloatSlider(value=0.17, min=0, max=1, step=0.01, description='spd gain')
steering_gain_slider = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description='steer gain')
steering_dgain_slider = widgets.FloatSlider(value=0.0, min=0, max=0.5, step=0.001, description='steer kd')
steering_bias_slider = widgets.FloatSlider(value=0.0, min=-0.3, max=0.3, step=0.01, description='steer bias')

startBtn = widgets.Button(description="Start", button_style='info')
pickupBtn = widgets.Button(description="Gripper Pick", button_style='success')
dropBtn   = widgets.Button(description="Gripper Drop", button_style='warning')

# Display
display(widgets.HBox([startBtn, pickupBtn, dropBtn]))
display(widgets.HBox([image_widget, y_slider, speed_slider]))
display(x_slider, steering_slider)
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

# ─────────────────────────────────────────────────────
# 4. 이벤트 핸들러 및 실행 제어
# ─────────────────────────────────────────────────────
finder = None
mover = None

def toggle(b):
    global finder, mover
    if b.description == "Start":
        finder = WorkingAreaFind(camera, image_widget)
        mover  = RobotMoving(camera, robot, model, device, mean, std,
                            x_slider, y_slider, image_widget,
                            steering_slider, speed_slider,
                            speed_gain_slider, steering_gain_slider,
                            steering_dgain_slider, steering_bias_slider)
        finder.start()
        mover.start()
        b.description = "Stop"; b.button_style = 'danger'
    else:
        finder.stop(); mover.stop()
        b.description = "Start"; b.button_style = 'info'

startBtn.on_click(toggle)
pickupBtn.on_click(lambda _: grip(gripper))
dropBtn.on_click(lambda _: release(gripper))

# ─────────────────────────────────────────────────────
# 5. S3 업로더 데몬 스레드 기동
# ─────────────────────────────────────────────────────
threading.Thread(target=start_s3_uploader, daemon=True).start()
print("✅ 모든 기능 준비 완료: Start → 주행 + 색상 인식 + Gripper + S3")
